<a href="https://colab.research.google.com/github/Aminfaraji/paddleocr_persian/blob/main/paddleocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !zip -r "/content/drive/MyDrive/paddel-ocr-shotor.zip" "/content/drive/MyDrive/paddel-ocr-shotor"

**Install Paddleocr compatible with the system (cuda11.2 & system linux)**


**download pretrained model this link**

https://drive.google.com/file/d/18wR7X2zDEdWBHBTPXSJT5S5L1_O7rmKh/view?usp=sharing

In [ ]:
!python -m pip install paddlepaddle-gpu==2.4.1.post112 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
%cd /content/PaddleOCR
!pip install -r requirements.txt
# !mkdir pretrained_model
# %cd pretrained_model/
# !wget https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/arabic_PP-OCRv3_rec_train.tar
# !tar -xf arabic_PP-OCRv3_rec_train.tar && rm -rf arabic_PP-OCRv3_rec_train.tar


**download dataset shotor from google drive**

https://drive.google.com/file/d/1kj8wQ45R63Gyp-Qv2mJaa1C4Y0u1JgEC/view?usp=sharing, https://drive.google.com/file/d/1zrZUJjeJgz0iAVeK8QmMZVAUdPDSpman/view?usp=sharing

In [ ]:
# !mkdir /content/PaddleOCR/train_data
# !unzip /content/drive/MyDrive/IDPL_train.zip -d /content/PaddleOCR/train_data
# !unzip /content/drive/MyDrive/IDPL_eval.zip -d /content/PaddleOCR/train_data
!unzip /content/drive/MyDrive/IDPL_eval.zip

In [5]:
import glob,pandas,shutil

a=glob.glob("/content/PaddleOCR/train_data/en_train_filtered/*.csv")
shutil.move(a[0],'/content/PaddleOCR/train_data')
# a=glob.glob("/content/PaddleOCR/train_data/en_val/*.csv")
# shutil.move(a[0],'/content/PaddleOCR/train_data')
# !rm -rf /content/PaddleOCR/train_data/en_val/labels.csv
# a

'/content/PaddleOCR/train_data/labels.csv'

**convert file .csv to .txt for train Paddleocr. (Paddleocr support of format .txt for train model)**

**recommend download file .txt**
https://drive.google.com/file/d/1-15OsG6mAxBgVXtjCE2bKVKk5Ry_Dcr3/view?usp=sharing,
*****************************
https://drive.google.com/file/d/1Q4FuTYAKjWtUgGBsAK0blYuSQm8Rme99/view?usp=sharing

In [ ]:
# !rm -rf /content/PaddleOCR/train_data/en_train_filtered/labels.csv
# !rm -rf /content/PaddleOCR/train_data/en_val/labels.csv
# !cp /content/drive/MyDrive/val.txt -r /content/PaddleOCR/train_data/
# !cp /content/drive/MyDrive/train.txt -r /content/PaddleOCR/train_data/

In [7]:
# # Read in the file
# with open('/content/drive/MyDrive/val.txt', 'r') as file :
#   filedata = file.read()

# # Replace the target string
# filedata = filedata.replace('.jpg', '.tif')

# # Write the file out again
# with open('/content/drive/MyDrive/val.txt', 'w') as file:
#   file.write(filedata)

# # Read in the file
# with open('/content/drive/MyDrive/train.txt', 'r') as file :
#   filedata = file.read()

# # Replace the target string
# filedata = filedata.replace('.jpg', '.tif')

# # Write the file out again
# with open('/content/drive/MyDrive/train.txt', 'w') as file:
#   file.write(filedata)
# 1. Open the CSV file in reading mode and the TXT file in writing mode
with open('/content/PaddleOCR/train_data/train.csv', 'r') as f_in, open('/content/PaddleOCR/train_data/train.txt', 'w') as f_out:

    # 2. Read the CSV file and store in variable
    content = f_in.read()

    # 3. Write the content into the TXT file
    f_out.write(content)
!rm -rf /content/PaddleOCR/train_data/train.csv
with open(r"/content/PaddleOCR/train_data/train.txt", 'r+') as fp:
    # read an store all lines into list
    lines = fp.readlines()
    # move file pointer to the beginning of a file
    fp.seek(0)
    # truncate the file
    fp.truncate()

    # start writing lines
    # iterate line and line number
    for number, line in enumerate(lines):
        # delete line number 5 and 8
        # note: list index start from 0
        if number not in [0]:
            fp.write(line)


In [8]:
import cv2 
import os 
#from imutils import perspective
import numpy as np 
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from imutils import perspective

**continue trained model**

In [2]:
%cd /content/PaddleOCR

/content/PaddleOCR


In [ ]:
!python3 tools/train.py -c /content/PaddleOCR/pretrained_model/arabic_PP-OCRv3_rec_train/config.yml -o Global.pretrained_model=/content/PaddleOCR/pretrained_model/arabic_PP-OCRv3_rec_train/best_accuracy

In [ ]:
!python3 tools/train.py -c /content/drive/MyDrive/output_IDPL/config.yml -o Global.checkpoints=/content/drive/MyDrive/output_IDPL/latest

Streaming output truncated to the last 5000 lines.
[2023/01/29 13:01:44] ppocr INFO: epoch: [1494/5000], global_step: 66470, lr: 0.000181, acc: 0.000000, norm_edit_dis: 0.601729, loss: 56.156654, avg_reader_cost: 0.00021 s, avg_batch_cost: 0.11401 s, avg_samples: 64.0, ips: 561.33126 samples/s, eta: 1 day, 14:11:15
[2023/01/29 13:01:45] ppocr INFO: epoch: [1494/5000], global_step: 66480, lr: 0.000181, acc: 0.000000, norm_edit_dis: 0.603575, loss: 55.561577, avg_reader_cost: 0.00020 s, avg_batch_cost: 0.11202 s, avg_samples: 64.0, ips: 571.31625 samples/s, eta: 1 day, 14:11:13
[2023/01/29 13:01:46] ppocr INFO: epoch: [1494/5000], global_step: 66490, lr: 0.000181, acc: 0.000000, norm_edit_dis: 0.607347, loss: 54.084984, avg_reader_cost: 0.00048 s, avg_batch_cost: 0.11510 s, avg_samples: 64.0, ips: 556.03050 samples/s, eta: 1 day, 14:11:11
[2023/01/29 13:01:47] ppocr INFO: epoch: [1494/5000], global_step: 66500, lr: 0.000181, acc: 0.000000, norm_edit_dis: 0.607528, loss: 54.013683, avg_re

In [ ]:
files=glob.glob("/content/drive/MyDrive/output_IDPL/*")
import os
for file in files:
  if file.find('230')<=0 and file.find('latest')<=0 and file.find('train')<=0 and file.find('best')<=0 and file.find('config')<=0:
    os.remove(file)

**evaluate dataset shotor**

In [ ]:
!python3 -m paddle.distributed.launch --gpus '0' tools/eval.py -c /content/drive/MyDrive/output/v3_arabic_mobile/config.yml -o Global.checkpoints=/content/drive/MyDrive/output/v3_arabic_mobile/best_accuracy

In [ ]:
%cd ..

/content/PaddleOCR


**test image uploaded on content**

In [4]:
!python3 tools/infer_rec.py -c /content/drive/MyDrive/output_IDPL/config.yml -o Global.pretrained_model=/content/drive/MyDrive/output_IDPL/best_accuracy  Global.infer_img=/content/امین.png

[2023/01/28 13:31:54] ppocr INFO: Architecture : 
[2023/01/28 13:31:54] ppocr INFO:     Backbone : 
[2023/01/28 13:31:54] ppocr INFO:         depth : [3, 6, 3]
[2023/01/28 13:31:54] ppocr INFO:         embed_dim : [64, 128, 256]
[2023/01/28 13:31:54] ppocr INFO:         img_size : [32, 100]
[2023/01/28 13:31:54] ppocr INFO:         last_stage : True
[2023/01/28 13:31:54] ppocr INFO:         local_mixer : [[7, 11], [7, 11], [7, 11]]
[2023/01/28 13:31:54] ppocr INFO:         mixer : ['Local', 'Local', 'Local', 'Local', 'Local', 'Local', 'Global', 'Global', 'Global', 'Global', 'Global', 'Global']
[2023/01/28 13:31:54] ppocr INFO:         name : SVTRNet
[2023/01/28 13:31:54] ppocr INFO:         num_heads : [2, 4, 8]
[2023/01/28 13:31:54] ppocr INFO:         out_channels : 192
[2023/01/28 13:31:54] ppocr INFO:         out_char_num : 120
[2023/01/28 13:31:54] ppocr INFO:         patch_merging : Conv
[2023/01/28 13:31:54] ppocr INFO:         prenorm : False
[2023/01/28 13:31:54] ppocr INFO:  

In [ ]:
files=glob.glob("/content/drive/MyDrive/output_IDPL/*")
for file in files:
  if file.find("iter_epoch_1079")<=0 and file.find("latest")<=0 and file.find("best")<=0 and file.find("train")<=0 and file.find("config")<=0:
    os.remove(file)

In [ ]:
# !wget https://paddleocr.bj.bcebos.com/rec_r45_abinet_train.tar
# !tar -xvf /content/rec_r45_abinet_train.tar
!pip install "paddleocr>=2.0.6" # 2.0.6 version is recommended
!pip install paddlepaddle


In [3]:
!paddleocr --image_dir /content/21098_2.tif --lang=fa

download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/Multilingual_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer/Multilingual_PP-OCRv3_det_infer.tar
100% 3.85M/3.85M [00:20<00:00, 190kiB/s]
download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/arabic_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/arabic/arabic_PP-OCRv3_rec_infer/arabic_PP-OCRv3_rec_infer.tar
100% 10.3M/10.3M [00:05<00:00, 1.83MiB/s]
download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar
100% 2.19M/2.19M [00:04<00:00, 522kiB/s]
[2023/01/29 22:31:38] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type=

In [10]:
# !wget https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/arabic_PP-OCRv3_rec_infer.tar
# !tar -xvf /content/arabic_PP-OCRv3_rec_infer.tar
!wget https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/arabic_PP-OCRv3_rec_infer.tar
# !wget https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/Multilingual_PP-OCRv3_det_infer.tar
# !tar -xvf /content/Multilingual_PP-OCRv3_det_infer.tar

--2023-01-29 22:10:21--  https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/arabic_PP-OCRv3_rec_infer.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10332160 (9.9M) [application/x-tar]
Saving to: ‘arabic_PP-OCRv3_rec_infer.tar’

arabic_PP-OCRv3_rec 100%[===================>]   9.85M  2.62MB/s    in 3.8s    

2023-01-29 22:10:27 (2.62 MB/s) - ‘arabic_PP-OCRv3_rec_infer.tar’ saved [10332160/10332160]

arabic_PP-OCRv3_rec_infer/
arabic_PP-OCRv3_rec_infer/inference.pdiparams
arabic_PP-OCRv3_rec_infer/inference.pdiparams.info
arabic_PP-OCRv3_rec_infer/inference.pdmodel


**Recommend:**

Please change the (configuration) file for train or evaluation.